In [1]:
!nvidia-smi

Mon Mar 08 18:49:30 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 456.71       Driver Version: 456.71       CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 106... WDDM  | 00000000:02:00.0  On |                  N/A |
|  0%   34C    P8     6W / 120W |    279MiB /  6144MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                                  |
|  GPU  

# Pre-Processing and Training the Model
This notebook goes through the steps in turning the cleaned data into a representation that can be used to train a pre-trained BERT model for sentiment analysis.

__Columns__
- `target` sentiment of tweet (0=positive, 1=negative)
- `tweet` cleaned tweet text

_Note_
_This notebook expects the zipped datasets to be in the ./resources sub-directory with the names train_twitter_data.zip, test_twitter_data, and validate_twitter_data (e.g., ./resources/train_twitter_data.zip). If the datasets do not exist open and run the `cleaning.ipynb` notebook to generate them._

In [2]:
import sys
!{sys.executable} -m pip install tensorflow-text
!{sys.executable} -m pip install tf-models-official

  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4835 sha256=37d183b05b3192f75f15d7b92fb70984f6ffd9dfdf9a23fced168d801f1cf9b4
  Stored in directory: c:\users\ryand\appdata\local\pip\cache\wheels\a0\16\9c\5473df82468f958445479c59e784896fa24f4a5fc024b0f501
  Created wheel for wrapt: filename=wrapt-1.12.1-cp38-cp38-win_amd64.whl size=33698 sha256=6f9a4b0ea873e07787c5f5af2055ccb831f8ecbf5299169c967d4c13eff556f1
  Stored in directory: c:\users\ryand\appdata\local\pip\cache\wheels\5f\fd\9e\b6cf5890494cb8ef0b5eaff72e5d55a70fb56316007d6dfe73
Successfully built termcolor wrapt
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.11.2
    Uninstalling wrapt-1.11.2:
      Successfully uninstalled wrapt-1.11.2
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16175 sha256=c2462338b0f82ac31bde2473bd322ccb9f2b16ea03af21780668fdcdc5594743
  Stored in directory: c:\users\ryand\appdata\local\pip\cache\wheels\ad\5c\ba\05fa33fa5855777

ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

tensorflow-metadata 0.28.0 requires absl-py<0.11,>=0.9, but you'll have absl-py 0.11.0 which is incompatible.


In [4]:
import os
import shutil

import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

from official.nlp import optimization

tf.get_logger().setLevel('ERROR')

## Read Files
Read the files into memory and convert them to TensorFlow datasets to begin pre-processing.

In [20]:
RESOURCES_DIR = 'resources'
train_file_name = 'train_twitter_data.zip'
test_file_name = 'test_twitter_data.zip'
validate_file_name = 'validate_twitter_data.zip'
column_dtypes = {'target': 'bool', 'tweet': 'object'}

train_df = pd.read_csv('./{}/{}'.format(RESOURCES_DIR, train_file_name), encoding = 'ISO-8859-1', compression='zip')
test_df = pd.read_csv('./{}/{}'.format(RESOURCES_DIR, test_file_name), encoding = 'ISO-8859-1', compression='zip')
validate_df = pd.read_csv('./{}/{}'.format(RESOURCES_DIR, validate_file_name), encoding = 'ISO-8859-1', compression='zip')

In [23]:
train_df.head()

#train_ds = tf.data.Dataset.from_tensor_slices((train_df['tweet'].values, train_df['label'].values))

#for feat, targ in train_ds.take(5):
  #print ('Features: {}, Target: {}'.format(feat, targ))

,tweets
0,how long you been on here is not too shabby yo...
1,i have drunk hickups i had a pretty laid back ...
2,yeahshe passed last week so sad
3,although the guy playing the dmeo is playing i...
4,no I am even more frustrated
